In [ ]:
#The following notebook produced the best scoring model for the Inception V3 Transfer Learning + Fine Tuning
#We decided to upload those notebooks to show the work behind the best models

In [ ]:
%cd ../input

In [ ]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import random

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import preprocess_input


from sklearn.utils import class_weight

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)



In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
# Dataset folders 
dataset_dir = 'dataset'
training_dir = os.path.join(dataset_dir, 'training')

In [ ]:
height = 299
width = 299

In [ ]:
#augmented + thresholding, validation has only preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_gen = ImageDataGenerator(validation_split=0.2,
                              fill_mode = 'constant',
                              rotation_range = 60,
                              horizontal_flip = True,
                              brightness_range = (0.8,1.2),
                              vertical_flip = True,
                              zoom_range = 0.1,
                              preprocessing_function = preprocess_input
                              )

train_gen = data_gen.flow_from_directory(directory = training_dir,
                                                 subset = 'training',
                                                 target_size=(height,width),
                                                 color_mode='rgb',
                                                 classes=None,
                                                 class_mode='categorical',
                                                 #batch_size=8,
                                                 shuffle=True,
                                                 seed=seed)

valid_gen = ImageDataGenerator(validation_split = 0.2,
                               preprocessing_function = preprocess_input).flow_from_directory(directory = training_dir,
                                                 subset = 'validation',
                                                 target_size=(height,width),
                                                 color_mode='rgb',
                                                 classes=None,
                                                 class_mode='categorical',
                                                 #batch_size=8,
                                                 shuffle=False,
                                                 seed=seed)
labels = np.unique(train_gen.classes)

In [ ]:
keys = range(14)
class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_gen.classes), 
                train_gen.classes)

weights = dict(zip(keys,class_weights))
print(weights)

In [ ]:
def get_next_batch(generator):
    batch = next(generator)

    image = batch[0]
    target = batch[1]

    print("(Input) image shape:", image.shape)
    print("Target shape:",target.shape)

    # Visualize only the first sample
    image = image[0]
    target = target[0]
    target_idx = np.argmax(target)
    print()
    print("Categorical label:", target)
    print("Label:", target_idx)
    print("Class name:", labels[target_idx])
    fig = plt.figure(figsize=(6, 4))
    plt.imshow(image)

    return batch

In [ ]:
_ = get_next_batch(valid_gen)


In [ ]:
input_shape = (299,299, 3)

In [ ]:
# Utility function to create folders and callbacks for training
from datetime import datetime

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('../working/Pasquale_InceptionV3')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                     save_weights_only=False, # True to save only weights
                                                     save_best_only=True) # True to save only the best epoch 
  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  # By default shows losses and metrics for both training and validation
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir, 
                                               profile_batch=0,
                                               histogram_freq=1)  # if > 0 (epochs) shows weights histograms
  callbacks.append(tb_callback)

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
  callbacks.append(es_callback)

  return callbacks

In [ ]:
# Apply same preprocessing used to train the supernet 
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
'''supernet = InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=input_shape
)'''
'''supernet.summary()'''
#tfk.utils.plot_model(supernet)

In [ ]:
'''# Use the supernet as feature extractor
supernet.trainable = False

#model with global avg pooling and dropout, way less parameters
inputs = tfk.Input(shape=input_shape)
x = supernet(inputs)

x = tfkl.GlobalAveragePooling2D()(x)

x = tfkl.Dense(
    512, 
    activation='relu',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)

x = tfkl.Dropout(0.3, seed=seed)(x)

x = tfkl.Dense(
    128, 
    activation='relu',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)

x = tfkl.Dropout(0.3, seed=seed)(x)

x = tfkl.Dense(
    32, 
    activation='relu',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)

x = tfkl.Dropout(0.3, seed=seed)(x)

outputs = tfkl.Dense(
    14, 
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)


# Connect input and output through the Model class
model = tfk.Model(inputs=inputs, outputs=outputs, name='model')
# Compile the model'''
'''model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy') #b4 optimizer = 'rmsprop'
model.summary()'''

In [ ]:
'''# Train the model
tf.get_logger().setLevel('ERROR')

callbacks = create_folders_and_callbacks(model_name='CNN_inception')
history = model.fit(
    x = train_gen,
    epochs = 50,
    validation_data = valid_gen, #don't use an augmented validation set
    callbacks = callbacks,
    class_weight = weights
).history'''

In [ ]:
'''model.save("../working/Pasquale_InceptionV3/InceptionV3_WOFT_v4_refined")'''

In [ ]:
model = tfk.models.load_model('finetuning/SubmissionModel')

In [ ]:
model.get_layer('inception_v3').trainable = False   
for layer in model.get_layer('inception_v3').layers[249:]:    
    if not isinstance(layer, tfk.layers.BatchNormalization):
        layer.trainable = True
for i, layer in enumerate(model.get_layer('inception_v3').layers):
    print(i, layer.name, layer.trainable)

In [ ]:
#best finetune trained with adam
lr = 0.0025
w_decay = 1e-06
optimizer=tfk.optimizers.Adam(learning_rate = lr,decay = w_decay)
model.compile(optimizer=optimizer, loss='kl_divergence', metrics = 'accuracy')


In [ ]:
def create_folders_and_callbacks(model_name):

    exps_dir = os.path.join('../working/Pasquale_InceptionV3')
    if not os.path.exists(exps_dir):
        os.makedirs(exps_dir)

    now = datetime.now().strftime('%b%d_%H-%M-%S')

    exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
    if not os.path.exists(exp_dir):
        os.makedirs(exp_dir)

    callbacks = []

    # Model checkpoint
    # ----------------
    ckpt_dir = os.path.join(exp_dir, 'ckpts')
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                     save_weights_only=False, # True to save only weights
                                                     save_best_only=True) # True to save only the best epoch 
    callbacks.append(ckpt_callback)

    # Visualize Learning on Tensorboard
    # ---------------------------------
    tb_dir = os.path.join(exp_dir, 'tb_logs')
    if not os.path.exists(tb_dir):
        os.makedirs(tb_dir)

    # By default shows losses and metrics for both training and validation
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir, 
                                               profile_batch=0,
                                               histogram_freq=1)  # if > 0 (epochs) shows weights histograms
    callbacks.append(tb_callback)

    # Early Stopping
    # --------------
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=17, restore_best_weights=True)
    callbacks.append(es_callback)
    
    #REDUCE LR ON PLATEAU
    
    lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss',factor = 0.2, verbose = 1, patience = 4)
    callbacks.append(lr_callback)



    return callbacks

In [ ]:
callbacks = create_folders_and_callbacks(model_name='Pasquale_InceptionV3_FineTuning')
history = model.fit(
    x = train_gen,
    epochs = 200,
    validation_data = valid_gen, #don't use an augmented validation set
    callbacks = callbacks,
    class_weight = weights
).history

In [ ]:
model.save("../working/Pasquale_InceptionV3/BestModelFT_FinalVersion")

In [ ]:
'''model_test_metrics = model.evaluate(test_gen, return_dict=True)
print("Test metrics with data augmentation")
print(model_test_metrics)'''